<h5>Alpide valid data words:</h5>

| Data Word   | Length (Bits) | Value (binary)                                  |
| :----------- | :------------- | :-----------------------------                   |
| REGION HEADER        | 8             | 111<region_id[4:0]>|
| IDLE        | 8             | 1111_1111                                       |
| Data Short  | 16            | 01<encoder_id[3:0]><addr[9:0]>                  |
| Data Long   | 24            | 00<encoder_id[3:0]><addr[9:0]>_0_<hit_map[6:0]> |

<h5> DATA SHORT </h5>
<p> encoder_id is the index of the priority encoder inside a region and addr is the pixel hit index from the pixel encoder. From part of the description in DATA LONG, it seems that the pixel with lowest address gets read out if there are multiple hits -- TODO CONFIRM</p>

<h5> DATA LONG </h5>
<p> only used if clustering is enabled - encoder_id and addr are the same as DATA SHORT, containing the geographical information of the first pixel (lowest address) the hit_map[6:0] contains the cluster shape information as a bitmap - a bit in the hit_map is set for any active pixel among the 7 after (based on PE addr) the one in the addr[9:0] field.</p>

<h5> Row and Column mappings: </h5>

```
{
    unsigned int pixel_x = MvtxDefs::getRow(hitkey);
	unsigned int pixel_z = MvtxDefs::getCol(hitkey);
}
```
| C0 | C1 |
| --- | ---|
| 0 | 1|
| 3 | 2|
|---|---|
| 4 | 5 |
| 7 | 6 |
 |  |

<p>In C0 of double column, the even rows have address: 2*row, and the odd rows have address (row+1)*2 -1. In C1 of double column, the even rows have address 2*row+1 and the odd rows have 2*row </p>

<h5> Clustering readout and DATA LONG </h5>

<h5> Data readout </h5>
<p> Region data frames are sent sequentially in ascending order - region header only comes from regions with pixelhit information </p>

In [1]:
EnableClustering = True 

In [22]:
print(f'{15:08b}')

00001111


In [2]:
import json
import numpy as np
import math

MVTXLayers = 3
ChipsPerStave = 9

datafd = open('Signal.json')
outfd = open('Signal_gbt.bin','w')
data = json.load(datafd)
print("Data Loaded")

Data Loaded


In [3]:
#index inside region
def pEncoderID(pixel_z):
    return (int(pixel_z/2) % 16)

def pEncoderRegion(pixel_z):
    return (int(pixel_z/32))

def isEven(num):
    if(num%2 == 0):
        return True
    else:
        return False

#this mapping is weird and I couldn't come up with a better model
def pixelAddr(row, column):
    doubleColIdx = (column % 2)
    #C0
    if(doubleColIdx == 0):
        if(isEven(row)):
            return 2*row
        else:
            return 2*(row+1) -1
    #C1
    else:
        if(isEven(row)):
            return 2*row + 1
        else:
            return 2*row
        return

#PEBlock are sets of 4 channels in the PE
#def rowCol(pixel):
#    PEBlock = int(pixel/4)
#    remainder = (pixel % 4)
#    #C1

#    if(remainder < 3):
#    #C2   
#    else()
        
def unique(x):
    x = np.array(x)
    return np.unique(x)    

In [5]:
class RawDataHeaderWord:
    ReservedHead = f'{0:08b}'
    ReservedHeadLarge = f'{0:032b}'
    ReservedMiddle = f'{0:016b}'
    
    #TODO
    #partially dfined for mvtx?
    DetectorField = f'{0:032b}'
    
    SourceID = f'{32:08b}'
    
    #TODO
    FEEID = f'{0:016b}'
    
    HeaderSize = f'{64:08b}'
    HeaderVersion = f'{8:08b}'
    
    #TODO
    #Trigger message BCO from GTM ?
    GTMBCO = f'{0:048b}'

    #TODO
    #Trigger message bunch crossing LHC clock from RU ?
    LHCBC = f'{0:012b}'
    
    #TODO
    # when 0x1 the packet is moved forward with priority?
    PriorityBit = f'{0:08b}'
    
    #TODO
    #Counter to keep track of CRU Data packet in same heartbeat
    PagesCounter = f'{0:016b}'
    
    #TODO
    #Trigger type bit set by Felix at HB trigger...
    TrgType= f'{0:032b}'
    
    def __init__(self, StopBit = 0):
        #TODO
        #1 if last page, 0 else
        self.StopBit = f'{StopBit:08b}'
    
    
        self.GBT0 = ReservedHead + DetectorField + SourceID + FEEID + HeaderSize + HeaderVersion 
        self.GBT1 = ReservedHead + GTMBCO + ReservedMiddle + LHCBC
        self.GBT2 = ReservedHeadLarge + PriorityBit + self.StopBit + PagesCounter + TrgType
        self.Data = GBT0 + GBT1 + GBT2 

In [6]:
class StartPacketWord:
    DV = f'{0:01b}'
    ControlCode = f'{1:04b}'
    Length = f'{0:016b}'
    TTSBusy = f'{0:016b}'
    Reserved = f'{0:044b}'
    Data = DV + ControlCode + Length + TTSBusy + Reserved
    
    def Print(self):
        print(Data)     

In [7]:
class EndPacketWord:
    DV = f'{0:01b}'
    ControlCode = f'{2:04b}'
    Length = f'{0:016b}'
    Checksum = f'{0:032b}'
    EndFlag = f'{0:01b}'
    Reserved = f'{0:027b}'
    Data = DV + ControlCode + Length + Checksum + EndFlag + Reserved
    
    def Print(self):
        print(Data)       

In [8]:
class SingleTransactionWord:
    DV = f'{0:01b}'
    ControlCode = f'{3:04b}'
    
    #expect 'Parameters' is already packaged in Bin format f'{num:075b}'
    def __init__(self, Parameters):
        self.Parameters = Parameters
        self.Data = DV + ControlCode + Parameters
        

In [9]:
class Hit:
    def __init__(self, TrkID, Layer, Stave, Chip, Pixel_x, Pixel_z):
        self.MVTXTrkID = TrkID
        self.Layer = Layer
        self.Stave = Stave
        self.Chip = Chip
        self.Pixel_x = Pixel_x
        self.Pixel_z = Pixel_z
        
    def Print(self):
        print("Hit Info-- Pixel_x: ", self.Pixel_x, " Pixel_z: ", self.Pixel_z, 
                  " pixelAddr: ", pixelAddr(self.Pixel_x,self.Pixel_z))

In [30]:
class RegionInfo:
    def __init__(self, PEHitList, Data, RegionNo):
        self.PEHitList = PEHitList
        self.Data = Data
        self.RegionNo = RegionNo

    def formatData(self):
                
    def Print(self):
        if self.isEmpty():
            return
        print("-------- Region: ", self.RegionNo, "--------")
        for PE, hits in enumerate(self.PEHitList):
            if hits:
                print("-------- Priority Encoder: " , PE , "--------")
                for hit in hits:
                    hit.Print()
                if(EnableClustering):
                    DATA_LONG = self.Data[PE]
                    DATA_SHORT = DATA_LONG[0]
                    ClusterMap = DATA_LONG[1]
                    print("ClusterMap: ", ClusterMap)
                else:
                    DATA_SHORT = self.Data[PE]
                print("DATA_SHORT: ", DATA_SHORT)
                print()
        
    def isEmpty(self):
        for PE in self.PEHitList:
            if len(PE) != 0:
               return False 
        return True     

In [31]:
class StaveInfo:
    def __init__(self, Layer, Stave, AlpideList):
        self.Layer = Layer
        self.Stave = Stave
        self.AlpideList = AlpideList.copy()
        
    def formatData(self):
        #TODO
        for alpide in self.AlpideList:
            alpide.formatData()
            alpideData = alpide.getData()
            
    def Print(self):
        print("-------- Stave Info --------")
        print( "Layer: ", self.Layer, " Stave: ", self.Stave, " Chip List... ")
        print()
        for Alpide in self.AlpideList:
            Alpide.Print()
        
class ReadoutUnit:
    def __init__(self, Stave):
        self.Stave = Stave

In [34]:
class AlpideInfo:
    def __init__(self,Layer,Stave,Chip,HitList):
        self.Layer = Layer
        self.Stave = Stave
        self.Chip = Chip
        self.HitList = HitList.copy()
        self.regionData = []
        self.binaryData=[]
        
    def getFormattedData(self):
        self.formatData()
        return self.binaryData
        
    def formatData(self):
        localRegionData=[]
        PEHitList = []
        if(self.HitList):
            #format by region (32)
            for i in range(0,32):
                regionHits = [x for x in self.HitList if pEncoderRegion(x.Pixel_z) == i]
                regionHits = sorted(regionHits, key = lambda hit : hit.Pixel_z)
                #format by priority Encoder (16 per region)
                for PEIdx in range(0,16):
                    PEHits = [x for x in regionHits if pEncoderID(x.Pixel_z) == PEIdx]
                    PEHits = sorted(PEHits, key = lambda hit: pixelAddr(hit.Pixel_x, hit.Pixel_z))
                    PEHitList.append(PEHits)
                    # build data words... 
                    if PEHits:
                        lowPixAddrHit = PEHits[0]
                        encoderID = pEncoderID(lowPixAddrHit.Pixel_z)
                        addr = pixelAddr(lowPixAddrHit.Pixel_x, lowPixAddrHit.Pixel_z)
                        DATA_SHORT = [encoderID, addr]
                        #build clusterHitMap
                        if(EnableClustering):
                            #hit map takes the 7 hits after the lowest pixel index
                            lowPixIdx = pixelAddr(lowPixAddrHit.Pixel_x, lowPixAddrHit.Pixel_z)
                            hitMap = 7*[0]
                            for idx, j in enumerate(range(lowPixIdx+1, lowPixIdx+8)):
                                #clumsy loop
                                #this has to happen only on the same priority encoder...
                                for hit in PEHits[1:]:
                                    if (pixelAddr(hit.Pixel_x, hit.Pixel_z) == j ):
                                        hitMap[idx] = 1
                                        break
                            #format hitmap as Little Endian 7 bit word
                            hitMap = int("".join(str(x) for x in hitMap),2)
                            hitMap = f'{hitMap:07b}'
                            hitMap = hitMap[::-1]
                            DATA_LONG = [DATA_SHORT,hitMap]
                    # empty alpides
                    else:
                        DATA_SHORT = []
                        if(EnableClustering):
                            DATA_LONG = [DATA_SHORT,'']
                    
                    if (EnableClustering):
                        localRegionData.append(DATA_LONG)   
                    else:
                        localRegionData.append(DATA_SHORT)
                
#                 print(PEHitList)
#                 print(localRegionData)
                self.regionData.append(RegionInfo(PEHitList.copy(), localRegionData.copy(), i))
                localRegionData.clear()
                PEHitList.clear()
            
            #now format the region data...
            for region in self.regionData():
                region.formatData()
                
            return 0
    
    def packageDataBinary(self):
        #TODO
        if(EnableClustering):
            return 0
        #TODO
        else:
            return 0    
    
    def Print(self):
        print("|------- Chip Info -------")
        print("Chip: ", self.Chip, " Region Hit Lists...")
        print()
        for region in self.regionData:
            region.Print()
        print()
                    
    ALPIDE_REGIONS = 32
    PENCODER_PER_REGION = 16
        


        

In [35]:
EventGBTinfo = []
AlpideData = []
StaveData = []
HitList = []
print("TotalEvents:", len(data['Events']))
print()

for evidx, ev in enumerate(data['Events']):
    
    print("--------------------")        
    print("--------------------")    
    print("Start of event: ", evidx)
    print("--------------------")
    print("--------------------")   
    print()
    
    EventGBTinfo.clear()
    AlpideData.clear()
    StaveData.clear()
    HitList.clear()
    RawHit = ev['RawHit']
    MVTXHits = RawHit['MVTXHits']
    
    # pull mvtx hit data info 
    for mvtxhit in MVTXHits:
        hitinfo = mvtxhit['ID']
        if(hitinfo['MVTXTrkID'] > 0):
            HitList.append(Hit(hitinfo['MVTXTrkID'],hitinfo['Layer'],hitinfo['Stave'],hitinfo['Chip'],
                                         hitinfo['Pixel_x'], hitinfo['Pixel_z']))
    
            
    # geographic grouping of Hits by Layer, Stave:
    for layer in range(0,MVTXLayers):
        layerHitList = [x for x in HitList if x.Layer == layer]
        stavesInLayer = unique([x.Stave for x in layerHitList])
        for stave in stavesInLayer:
            layerStaveHitList = [x for x in layerHitList if x.Stave == stave]
            # reduce data on ChipID 
            for ChipNo in range(0,ChipsPerStave):
                ChipHits = [x for x in layerStaveHitList if x.Chip == ChipNo]
                if ChipHits:
                    SampleHit = ChipHits[0]
                    AlpideData.append(AlpideInfo(SampleHit.Layer,SampleHit.Stave,SampleHit.Chip,
                                         ChipHits))
            StaveData.append(StaveInfo(layer,stave,AlpideData))
            AlpideData.clear()

            
    # format 9 alpide worth of data for each stave for sending to RU     
    for stave in StaveData:
        stave.formatData()
        stave.Print()
        
    print("------------")        
    print("------------")    
    print("end of event")
    print("------------")
    print("------------")
    print()
    break

TotalEvents: 170

--------------------
--------------------
Start of event:  0
--------------------
--------------------

-------- Stave Info --------
Layer:  0  Stave:  1  Chip List... 

|------- Chip Info -------
Chip:  6  Region Hit Lists...

-------- Region:  2 --------
-------- Priority Encoder:  4 --------
Hit Info-- Pixel_x:  260  Pixel_z:  72  pixelAddr:  520
Hit Info-- Pixel_x:  261  Pixel_z:  73  pixelAddr:  522
Hit Info-- Pixel_x:  261  Pixel_z:  72  pixelAddr:  523
Hit Info-- Pixel_x:  262  Pixel_z:  72  pixelAddr:  524
Hit Info-- Pixel_x:  262  Pixel_z:  73  pixelAddr:  525
ClusterMap:  0011110
DATA_SHORT:  [4, 520]

-------- Priority Encoder:  5 --------
Hit Info-- Pixel_x:  261  Pixel_z:  74  pixelAddr:  523
ClusterMap:  0000000
DATA_SHORT:  [5, 523]


-------- Stave Info --------
Layer:  0  Stave:  2  Chip List... 

|------- Chip Info -------
Chip:  2  Region Hit Lists...

-------- Region:  13 --------
-------- Priority Encoder:  11 --------
Hit Info-- Pixel_x:  21  Pix

In [ ]:

print(pE)

print(pixelAddr(260,72))
print(pixelAddr(261,73))
print(pixelAddr(261,74))
print(pixelAddr(261,72))
print(pixelAddr(262,72))
print(pixelAddr(262,73))



print()


print (pixelAddr(21,438))
print (pixelAddr(21,439))
print (pixelAddr(22,439))
print (pixelAddr(22,438))

In [ ]:
print(pixelAddr(261,73))
print(pixelAddr(261,74))

In [ ]:
columns = range(0,1024)
encoderIds = [pEncoderID(x) for x in columns]
#print (columns)
#print (encoderIds)

#print(pixelAddr(0,1))

for i in range(0,512):
    for j in range(0,2):
        print('{:0>4}'.format(pixelAddr(i,j)) , end=" ")
    print()
    


In [ ]:
a = 7*[0]
a[1] = 1
a[2] = 